# Create database of mutations

In [1]:
import os
from collections import defaultdict
import gzip

import bgreference
import numpy as np
import pandas as pd

#### Input files

In [2]:
boostdm_dir = './../boostDM/boostDM-intogen-prediction/'
intogen_drivers_f = './../intogen/IntOGen-Drivers-20200201_Compendium_Cancer_Genes.tsv'
intogen_negative_f = './../intogen/intOGen-20191022_Negative_gene_set.tsv'
vep_dir = '/workspace/projects/intogen_2017/runs/20200102/vep'
cgi_biomarkers_f = './../CGI/selected_biomarkers.tsv'

#### Output file

In [3]:
output_file = './mutations_db.tsv.gz'

#### Other input data

In [4]:
cancer_types = {
    'skin': ['CM', 'SCCC', 'SBCC'], 
    'lung': ['NSCLC', 'LUSC', 'LUAD', 'SCLC']
}

In [5]:
cancer_types = dict([(ct_specific, ct_general) for ct_general, v in cancer_types.items() for ct_specific in list(v)])

In [6]:
cancer_types

{'CM': 'skin',
 'SCCC': 'skin',
 'SBCC': 'skin',
 'NSCLC': 'lung',
 'LUSC': 'lung',
 'LUAD': 'lung',
 'SCLC': 'lung'}

## Mutations DB

In [7]:
# Object to load
mutations_info = defaultdict(lambda: defaultdict(dict))

#### 1) Load CGI biomarkers

In [8]:
biomarkers_df = pd.read_csv(cgi_biomarkers_f, sep='\t', header=0)

In [9]:
biomarkers_df['cancer_type_general'] = biomarkers_df.apply(lambda x: cancer_types.get(x['Primary Tumor type'], 'NAN'), axis=1)

In [10]:
biomarkers_df.head()

,Biomarker,Gene,Alteration,Drug,Primary Tumor type,Inhibitor type,Approved,cancer_type_general
0,ALK_E1408V,ALK,E1408V,Brigatinib,LUAD,Pan-TK inhibitor,False,lung
1,ALK_L1196M,ALK,L1196M,Brigatinib,LUAD,Pan-TK inhibitor,False,lung
2,ALK_S1206Y,ALK,S1206Y,Ceritinib,LUAD,ALK inhibitor,False,lung
3,ALK_G1269A,ALK,G1269A,Ceritinib,LUAD,ALK inhibitor,False,lung
4,ALK_I1171T,ALK,I1171T,Ceritinib,LUAD,ALK inhibitor,False,lung


In [11]:
len(biomarkers_df['Biomarker'].unique())

55

In [12]:
len(biomarkers_df.loc[biomarkers_df['Approved'] == True]['Biomarker'].unique())

2

#### 2) Get driver mutations from boostDM

Comments: 
    
- Truncating mutations are skipped
- Mutations with AA change not available (nan) are skipped
- Biomarkers with unkwown drug are labeled as therapy 'None'

In [13]:
# Identify cohorts with the cancer_types of interest
for file in os.scandir(boostdm_dir): 
    if file.name.split('.')[1] in set(cancer_types.keys()): 
        
        # Cancer type in data 
        cancer_type_specific = file.name.split('.')[1]
        cancer_type_general = cancer_types[cancer_type_specific]
        
        # Read df
        df = pd.read_csv(file.path, sep='\t', header=0)
        
        # Subset driver mutations
        df_drivers = df.loc[df['boostDM_class'] == True].copy()
        median_boostdm_score = df_drivers['boostDM_score'].describe()['50%']
        
        # Save information
        for _, row in df_drivers.iterrows(): 
            
            # If missense mutation 
            # If BoostDM score above the median (higher driver evidence)
            if row['csqn_type_missense'] and row['boostDM_score'] >= median_boostdm_score: 
                
                # If AA change is not nan
                if row['aachange'] == row['aachange']: 
                                                                      
                    # Mutation id (BRAF_V600E)
                    mutation_id = f"{row['gene']}_{row['aachange']}"
                    
                    # Gene
                    mutations_info[cancer_type_general][mutation_id]['gene'] = row['gene']
                    
                    # Cancer type
                    mutations_info[cancer_type_general][mutation_id]['cancer_type_specific'] = cancer_type_specific
                    mutations_info[cancer_type_general][mutation_id]['cancer_type_general'] = cancer_type_general
                    
                    # AA change
                    mutations_info[cancer_type_general][mutation_id]['aachange'] = row['aachange']
                   
                    # Nucleotide change
                    chrom = row['chr']
                    pos = row['pos']
                    ref = bgreference.refseq('hg38', chrom, pos, 1)
                    alt = row['alt']
                    mutations_info[cancer_type_general][mutation_id]['nuchange'] = f'chr{chrom}:{pos}_{ref}>{alt}'
                    
                    # Driver or passenger
                    mutations_info[cancer_type_general][mutation_id]['driver_passenger'] = 'driver'
                    
                    # Oncogene or tumor suppresor gene
                    if row['role_Act'] == 1: 
                        role = 'og'
                    elif row['role_LoF'] == 1: 
                        role = 'tsg'
                    else:
                        role = 'unknown'
                    mutations_info[cancer_type_general][mutation_id]['og_tsg'] = role
                    
                    # Targeted therapy
                    # Select mutation in the cancer type
                    biomarker_info_df = biomarkers_df.loc[
                        (biomarkers_df['Biomarker'] == mutation_id) & 
                        (biomarkers_df['cancer_type_general'] == cancer_type_general)
                    ]
                    # If mutation is biomarker
                    # Drug is approved or not
                    if len(biomarker_info_df) > 0: 
                        mutations_info[cancer_type_general][mutation_id]['targeted_therapy'] = biomarker_info_df['Drug'].iloc[0]
                        mutations_info[cancer_type_general][mutation_id]['targeted_therapy_approved'] = biomarker_info_df['Approved'].iloc[0]

                    else: 
                        mutations_info[cancer_type_general][mutation_id]['targeted_therapy'] = 'None'
                        mutations_info[cancer_type_general][mutation_id]['targeted_therapy_approved'] = False

In [14]:
len(mutations_info['lung']) # 5334

529

In [15]:
len(mutations_info['skin']) #3642

515

#### 3) Get passenger mutations from negative genes in intOGen

Comments: 
    
- Passengers OG or TSG role: unkwnown

In [16]:
# New names for cancer types are required
cancer_types_v2 = {
    'SKCM': 'skin', 
    'LUSC': 'lung', 
    'LUAD': 'lung'
}

In [17]:
# Read negatives df
negatives_df = pd.read_csv(intogen_negative_f, sep='\t', header=None)

for cancer_type_specific, cancer_type_general in cancer_types_v2.items(): 
    
    # Number of passenger genes per cancer type
    total_number_passenger_genes = 100 if cancer_type_specific == 'SKCM' else 50
    
    # Get VEP info
    # Get only missense
    vep_f = os.path.join(vep_dir, f'TCGA_WXS_{cancer_type_specific}.out.gz')
    vep_df = pd.read_csv(vep_f, sep='\t', header=0)
    vep_df = vep_df.loc[vep_df['Consequence'] == 'missense_variant'].copy()
    vep_genes = set(vep_df['SYMBOL'].unique())
    
    # Get cancer type negative genes
    # Get half of passenger genes from olfactory receptors and the other half from non olfactory receptors
    negatives_ctype_df = negatives_df.loc[negatives_df[0] == cancer_type_specific].copy()
    negative_genes = set(negatives_ctype_df[1].iloc[0].split(','))
    # Split ORs and non-ORs
    olfactory_receptors = set([gene for gene in negative_genes if gene.startswith('OR')])
    non_olfactory_receptors = negative_genes.difference(olfactory_receptors)
    # Intersect with genes in VEP
    olfactory_receptors.intersection_update(vep_genes)
    non_olfactory_receptors.intersection_update(vep_genes)
    # Randomly choose the desired number
    olfactory_receptors = np.random.choice(list(olfactory_receptors), total_number_passenger_genes // 2)
    non_olfactory_receptors = np.random.choice(list(non_olfactory_receptors), total_number_passenger_genes // 2)
    
    # Iterate through genes
    # Save one mutation per gene
    for gene in list(olfactory_receptors) + list(non_olfactory_receptors): 
        # Get first mutation
        gene_vep_df = vep_df.loc[vep_df['SYMBOL'] == gene].iloc[0]

        # AA change
        prot_pos = gene_vep_df['Protein_position']
        aa_ref, aa_alt = gene_vep_df['Amino_acids'].split('/')
        aachange = aa_ref + prot_pos + aa_alt

        # Mutation id (BRAF_V600E)
        mutation_id = f"{gene}_{aachange}"

        # Gene
        mutations_info[cancer_type_general][mutation_id]['gene'] = gene

        # Cancer type
        mutations_info[cancer_type_general][mutation_id]['cancer_type_specific'] = cancer_type_specific
        mutations_info[cancer_type_general][mutation_id]['cancer_type_general'] = cancer_type_general

        # AA change
        mutations_info[cancer_type_general][mutation_id]['aachange'] = aachange

        # Nucleotide change
        chrom_pos = gene_vep_df['Location']
        _, _, ref, alt = gene_vep_df['#Uploaded_variation'].split('__')
        mutations_info[cancer_type_general][mutation_id]['nuchange'] = f'chr{chrom_pos}_{ref}>{alt}'

        # Driver or passenger
        mutations_info[cancer_type_general][mutation_id]['driver_passenger'] = 'passenger'

        # Oncogene or tumor suppresor gene
        mutations_info[cancer_type_general][mutation_id]['og_tsg'] = 'unknown'

        # Targeted therapy
        # Select mutation in the cancer type
        mutations_info[cancer_type_general][mutation_id]['targeted_therapy'] = 'None'

        # Drug is approved or not
        mutations_info[cancer_type_general][mutation_id]['targeted_therapy_approved'] = False

/home/carnedo/anaconda3/envs/hotspots_framework/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
len(mutations_info['lung']) 

628

In [19]:
len(mutations_info['skin']) 

611

#### 4) Save table

In [20]:
header = [
    'cancer_type',
    'mutation_id',
    'gene', 
    'aa_change', 
    'dna_change', 
    'driver_passenger', 
    'og_tsg', 
    'targeted_therapy',
    'targeted_therapy_approved'
]

In [21]:
with gzip.open(output_file, 'wt') as ofd: 
    ofd.write('{}\n'.format('\t'.join(header)))
    for cancertype, mutations in mutations_info.items(): 
        for mutation_id, data in mutations.items(): 
            # FIXME targeted therapy
            ofd.write('{}\n'.format('\t'.join([
                cancertype, 
                mutation_id, 
                data['gene'],
                data['aachange'],
                data['nuchange'],
                data['driver_passenger'],
                data['og_tsg'],
                data['targeted_therapy'], 
                str(data['targeted_therapy_approved']),
            ])))

In [22]:
mutations_df = pd.read_csv(output_file, sep='\t', header=0)

In [23]:
mutations_df.head()

,cancer_type,mutation_id,gene,aa_change,dna_change,driver_passenger,og_tsg,targeted_therapy,targeted_therapy_approved
0,lung,FBXW7_R505P,FBXW7,R505P,chr4:152326136_C>G,driver,tsg,None,False
1,lung,FAM135B_G186E,FAM135B,G186E,chr8:138243054_C>T,driver,og,None,False
2,lung,ERBB3_C331R,ERBB3,C331R,chr12:56088750_T>C,driver,og,None,False
3,lung,PTEN_R130G,PTEN,R130G,chr10:87933147_C>G,driver,tsg,None,False
4,lung,PTEN_C136R,PTEN,C136R,chr10:87933165_T>C,driver,tsg,None,False


In [24]:
len(mutations_df.loc[mutations_df['targeted_therapy_approved'] == True]['mutation_id'].unique())

1